In [1]:
# !pip install requests-html

In [2]:
"https://www.baseball-reference.com/leagues/daily.fcgi?request=1&type=b&dates=lastndays&lastndays=7&since=2021-04-01&fromandto=2021-04-01.2021-04-30&level=mlb&franch=ANY"

'https://www.baseball-reference.com/leagues/daily.fcgi?request=1&type=b&dates=lastndays&lastndays=7&since=2021-04-01&fromandto=2021-04-01.2021-04-30&level=mlb&franch=ANY'

In [3]:
#finding
from bs4 import BeautifulSoup
import requests
import numpy as np
def getURLs(l_rows):
    dictRet=dict()#each players game log is stored
    col=0

    for r in l_rows:
        col=0
        for c in r.find_all("td"):        
            col+=1
            if col==1:
                playerName=c.text 
            if c.text.strip()=='gl':# find gameLog link in gl href
                url=c("a")[0]['href']
                dictRet[playerName]=url#each players game log is stored
                break
    
    return dictRet

def getPastDayStats(days):
    
    #URL="https://www.baseball-reference.com/leagues/daily.fcgi?request=1&type=b&dates=lastndays&lastndays=7&since=2021-04-01&fromandto=2021-04-01.2021-04-30&level=mlb&franch=ANY"
    URL="https://www.baseball-reference.com/leagues/daily.fcgi?request=1&type=b&dates=lastndays&lastndays="+str(days)+"&since=2021-04-01&fromandto=2021-04-01.2021-04-30&level=mlb&franch=ANY"
    print(URL)
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, 'html.parser')        
    results=soup.find(id="daily")
    cells=results.find_all("td")
    headers=results.find_all("thead")
    i=0
    headList=[]

    for h in headers:#just find what each col refers to
        for cells in h.text.split('\n'):
            if cells.strip() != '' and cells.strip() != 'Rk':
                i+=1        
                headList.append(cells)
    print(headList)

    rows=results.find_all("tr")
    statDict=dict()

    for r in rows:
        colCount=0
        statListTemp=[]
        cols=r.find_all("td")
        for c in cols:
            if c.text.strip()!='gl':#skip gl for now
                colCount+=1            
            if colCount==1:
                keyReq=c.text.strip()#using player-name based dictionary
                colCount+=1
            elif colCount>1 and c.text.strip()!='gl':
                statListTemp.append(c.text.strip())#input respective stats
        if colCount>1:
            statDict[keyReq]=statListTemp#put into dict

    urlGLDict=getURLs(rows)
    
    return statDict, urlGLDict, headList

def getNumpyAggregate(statDict,headList, categoryReq, topK):
    
#     mappingList=['G', 'PA', 'AB', 'R', 'H', '2B', '3B', 'HR', 'RBI', 'BB', 'IBB', 'SO', 'HBP', 'SH', 'SF', 'GDP', 'SB', 'CS', 'BA', 'OBP', 'SLG', 'OPS']
    mappingList=headList[5:]    
    mappingDict=dict()

    for i, m in enumerate(mappingList):
        mappingDict[m]=i

    statArray=np.zeros((0,len(headList)-5), dtype=np.float32)
    temp=[]
    for v in statDict.values():
        for i in v[4:]:
            if i != '':
                temp.append(float(i))
            else:
                temp.append(0)
        statArray=np.vstack((statArray, np.array(temp, dtype=np.float32)))
        temp=[]

    category=mappingDict[categoryReq]

    dictIndices=np.argpartition(statArray[:,category],-topK)[-topK:]#get players with top 20 hits

    playersToCheck=[]
    for d in dictIndices:
        playersToCheck.append(list(statDict.keys())[d])
        
    return statArray, playersToCheck

def makeNumpyGL(headList, statDict, batter=True,):
    #only getting 'PA','AB','R','H','2B','3B','HR','RBI','BB','IBB','SO','HBP','SH','SF','GDP','SB','CS'
    
    statArray=np.zeros((0,len(headList)-7), dtype=np.float32)#took out string columns
    temp=[]
    for v in statDict.values():
        for i in v[4:-2]:
            if i != '':
                temp.append(float(i))
            else:
                temp.append(0)
        
        statArray=np.vstack((statArray, np.array(temp, dtype=np.float32)))
        temp=[]
        
    return statArray

def getGLBatter(urlReq):
    stats=requests.get(urlReq)
    stats=BeautifulSoup(stats.content, 'html.parser')
    statTable=stats.find(id="batting_gamelogs_milb")

    headersGL=['DatePlayed','Lev','Tm','vs','Opp','PA','AB','R','H','2B','3B','HR',
               'RBI','BB','IBB','SO','HBP','SH','SF','GDP','SB','CS','Pos','E']
    statDictGL=dict()

    colNo=0
    for table in statTable.find_all('tbody'):#table body

        for row in table.find_all('tr'):#per row  
            colNo=0
            for col in row.find_all('td'): 
                if colNo==0:
                    keyReq=col.text
                    statDictGL[keyReq]=[]
                if colNo>0:
                    if col.text.strip()!='':
        #                 print('colNo:', colNo, col['data-stat'])
                        statDictGL[keyReq].append(col.text)

                    else:
        #                 print('colNo:', colNo, col['data-stat'])
                        if col.text.strip=='@':
                            statDictGL[keyReq].append(0)                    
                        else:
                            statDictGL[keyReq].append(1)
                colNo+=1
                    
    return statDictGL, makeNumpyGL(headersGL, statDictGL)
#     print(list(statDict.keys())[d],statDict[list(statDict.keys())[d]])

In [4]:
#['G', 'PA', 'AB', 'R', 'H', '2B', '3B', 'HR', 'RBI', 'BB', 'IBB', 'SO', 'HBP', 'SH', 'SF', 'GDP', 'SB', 'CS', 'BA', 'OBP', 'SLG', 'OPS']['G', 'PA', 'AB', 'R', 'H', '2B', '3B', 'HR', 'RBI', 'BB', 'IBB', 'SO', 'HBP', 'SH', 'SF', 'GDP', 'SB', 'CS', 'BA', 'OBP', 'SLG', 'OPS']
statDict2, urlGLDict2, headList2 = getPastDayStats(30)
statArray, playersToCheck=getNumpyAggregate(statDict2,headList2, 'RBI', 100)#find top K in the mentioned stat

https://www.baseball-reference.com/leagues/daily.fcgi?request=1&type=b&dates=lastndays&lastndays=30&since=2021-04-01&fromandto=2021-04-01.2021-04-30&level=mlb&franch=ANY
['Name', 'Age', '#days', 'Lev', 'Tm', 'G', 'PA', 'AB', 'R', 'H', '2B', '3B', 'HR', 'RBI', 'BB', 'IBB', 'SO', 'HBP', 'SH', 'SF', 'GDP', 'SB', 'CS', 'BA', 'OBP', 'SLG', 'OPS']


In [5]:
#might not need top K prediction for inference we can directly input playersToCheck from a list

In [6]:
headersGL=['DatePlayed','Lev','Tm','vs','Opp','PA','AB','R','H','2B','3B','HR',
               'RBI','BB','IBB','SO','HBP','SH','SF','GDP','SB','CS','Pos','E']
hh=['Name', 'Age', '#days', 'Lev', 'Tm', 'G', 'PA', 'AB', 'R', 'H', '2B', '3B', 'HR', 'RBI', 'BB', 'IBB', 'SO',
 'HBP', 'SH', 'SF', 'GDP', 'SB', 'CS', 'BA', 'OBP', 'SLG', 'OPS']

topKBatting=[]
for p in playersToCheck:    
    topKBatting.append([p,np.array(statDict2[p][5:-4],dtype=np.float32)])

topKBatting=np.array(topKBatting)#got batting stats inference

In [7]:
topKBatting

array([['Trea Turner',
        array([124., 113.,  15.,  32.,   5.,   0.,   1.,  12.,  10.,   0.,  24.,
         0.,   0.,   1.,   0.,   6.,   3.], dtype=float32)],
       ['Lourdes Gurriel Jr.',
        array([93., 87., 13., 31.,  7.,  0.,  4., 12.,  4.,  0., 22.,  0.,  1.,
        1.,  1.,  0.,  0.], dtype=float32)],
       ['Gregory Polanco',
        array([96., 84.,  7., 17.,  2.,  1.,  3., 12.,  8.,  1., 30.,  0.,  0.,
        4.,  0.,  1.,  0.], dtype=float32)],
       ['Gleyber Torres',
        array([92., 82., 10., 27.,  4.,  0.,  2., 13.,  8.,  0., 14.,  0.,  1.,
        1.,  1.,  2.,  1.], dtype=float32)],
       ['Yermin Mercedes',
        array([106.,  95.,   9.,  18.,   2.,   0.,   2.,  12.,   9.,   0.,  16.,
         1.,   0.,   1.,   1.,   0.,   0.], dtype=float32)],
       ['Martin Maldonado',
        array([80., 72., 10., 14.,  2.,  0.,  2., 12.,  7.,  0., 19.,  1.,  0.,
        0.,  0.,  0.,  0.], dtype=float32)],
       ['Patrick Wisdom',
        array([56., 52., 11.

In [8]:
# 'https://swishanalytics.com/optimus/mlb/batter-vs-pitcher-stats?date='+dateReq

In [9]:
#getting batter vs pitcher stats from website
from datetime import datetime
dateReq= datetime.today().strftime('%Y-%m-%d')

listBvsP=[]

print('finding for : ', dateReq)

urlReq='https://swishanalytics.com/optimus/mlb/batter-vs-pitcher-stats?date='+dateReq
page=requests.get(urlReq)

soup = BeautifulSoup(page.content, 'html.parser')        

pgResults=str(soup.find_all('script'))

#finding way to just get the json type response in our own way
ss='this.playerArray = '
i=pgResults.find(ss)
i2=pgResults.find('}]')
i2
dictList=list(eval(pgResults[i+len(ss)+1:i2+len('}')]))
bVsp=dict()
pitcherInfo=[]
for p in playersToCheck:        
    pitcherReq=None
    for b in dictList:
        if b['batter']==p.replace("é","e") or b['batter']==p.replace("í","i") or b['batter']==p.replace("Á","A"):        
            pitcherReq=b #store required dictionary
            pitcherInfo.append([b['batter'], b['pitcher'], dateReq])
            break    
            
    if pitcherReq==None:
        print('todays matchup for ', p, 'not found')
        continue
        
    bVsPHeaders=['pa', 'ab', 'h', 'singles', 'doubles', 
      'triples', 'hr', 'bb', 'so', 'batting_avg', 'obp', 'slugging_pct']

    bVsp[p]=np.zeros(len(bVsPHeaders))  #initialize pitching stats array #second key will be player

    for hNo, h in enumerate(bVsPHeaders):         
        bVsp[p][hNo]=pitcherReq[h]

#         listBvsP.append(bVsp)#for each batter add to the list
    print('done: ', p)  

bVsp

finding for :  2021-06-14
done:  Trea Turner
done:  Lourdes Gurriel Jr.
done:  Gregory Polanco
todays matchup for  Gleyber Torres not found
done:  Yermin Mercedes
todays matchup for  Martin Maldonado not found
done:  Patrick Wisdom
done:  Carlos Santana
todays matchup for  Gary Sanchez not found
done:  Robbie Grossman
done:  Jake Fraley
todays matchup for  Yordan Álvarez not found
done:  Maikel Franco
todays matchup for  Jose Altuve not found
done:  Eric Haase
todays matchup for  William Contreras not found
done:  Harold Ramirez
done:  Albert Pujols
done:  Christian Vazquez
done:  Kyle Seager
done:  Charlie Blackmon
done:  Miguel Cabrera
done:  Joey Wendle
done:  Whit Merrifield
done:  Bobby Dalbec
done:  Brandon Lowe
done:  Nick Castellanos
done:  Christian Yelich
done:  Billy McKinney
done:  Matt Chapman
done:  Pete Alonso
done:  Jared Walsh
todays matchup for  Freddie Freeman not found
done:  Javier Baez
done:  Ian Happ
done:  Justin Upton
todays matchup for  Adolis García not found

{'Trea Turner': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 'Lourdes Gurriel Jr.': array([7.   , 5.   , 3.   , 3.   , 0.   , 0.   , 0.   , 2.   , 1.   ,
        0.6  , 0.714, 0.6  ]),
 'Gregory Polanco': array([21.   , 17.   ,  2.   ,  2.   ,  0.   ,  0.   ,  0.   ,  4.   ,
         9.   ,  0.118,  0.286,  0.118]),
 'Yermin Mercedes': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 'Patrick Wisdom': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 'Carlos Santana': array([36.   , 33.   ,  8.   ,  4.   ,  3.   ,  0.   ,  1.   ,  2.   ,
         1.   ,  0.242,  0.306,  0.424]),
 'Robbie Grossman': array([4.  , 4.  , 1.  , 0.  , 1.  , 0.  , 0.  , 0.  , 1.  , 0.25, 0.25,
        0.5 ]),
 'Jake Fraley': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 'Maikel Franco': array([1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 'Eric Haase': array([2. , 2. , 1. , 1. , 0. , 0. , 0. , 0. , 1. , 0.5, 0.5, 0.5]),
 'Harold Ramirez': array([0., 0., 0., 0., 0

In [10]:
topKBatting[0,1].shape[0],list(bVsp.values())[0].shape[0],len(list(bVsp.keys()))

(17, 12, 83)

# == PITCHER SECTION===#

In [11]:










































def getPastDayStatsPitcher():
    
    #URL="https://www.baseball-reference.com/leagues/daily.fcgi?request=1&type=b&dates=lastndays&lastndays=7&since=2021-04-01&fromandto=2021-04-01.2021-04-30&level=mlb&franch=ANY"
    URL='https://www.baseball-reference.com/leagues/daily.fcgi?request=1&type=p&lastndays=7&dates=since&since=2021-04-01&fromandto=2021-05-01.2021-05-31&level=mlb&franch=ANY'
    print(URL)
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, 'html.parser')        
    results=soup.find(id="daily")
    cells=results.find_all("td")
    headers=results.find_all("thead")
    i=0
    headList=[]
    heads=['Name', 'gl', 'dummy', 'Age', '#days', 'Lev', 'Tm', 'G', 'GS', 'W', 'L', 'SV', 'IP', 'H', 
       'R', 'ER', 'BB', 'SO', 'HR', 'HBP', 'ERA', 'AB', '2B', '3B', 'IBB', 'GDP', 'SF', 
       'SB', 'CS', 'PO', 'BF', 'Pit', 'Str', 'StL', 'StS', 'GB/FB', 'LD', 'PU', 'WHIP', 'BAbip', 'SO9', 'SO/W']
    
    headersCounted=['W','IP','H', 'R', 'ER', 'BB', 'SO', 'HR', 'HBP', 'AB', '2B', '3B', 'IBB','Pit', 'Str', 'StL', 'StS']
    mapping=dict()    
    
    for i, h in enumerate(heads):
        mapping[i]=h

    for h in headers:#just find what each col refers to
        for cells in h.text.split('\n'):
            if cells.strip() != '' and cells.strip() != 'Rk':
                i+=1        
                headList.append(cells)
    print(headList)

    rows=results.find_all("tr")
    statDict=dict()

    for r in rows:
        colCount=0
        statListTemp=[]
        cols=r.find_all("td")        
        for c in cols:

            if colCount>len(heads)-1:
                break
            if mapping[colCount] not in headersCounted and colCount>1: #colCount>1 to make keyReq
                colCount+=1
                continue
            else:

                if c.text.strip()!='gl':#skip gl for now
                    colCount+=1            
                if colCount==1:
                    keyReq=c.text.strip()#using player-name based dictionary
                    colCount+=1
                elif colCount>1 and c.text.strip()!='gl':
                    if '%' in c.text.strip(): #in case a percent sign
                        statListTemp.append(c.text.strip()[:-1])#input upto % sign in required stats
                    elif c.text.strip()=='':
                        statListTemp.append(0)#input respective stats
                    else:
                        statListTemp.append(c.text.strip())#input respective stats
                    
          
        if colCount>1:
            statDict[keyReq]=statListTemp#put into dict
            
    urlGLDict=getURLs(rows)
    
    return statDict, urlGLDict, headList

In [12]:
pDict, urlDictP, headListP = getPastDayStatsPitcher()

https://www.baseball-reference.com/leagues/daily.fcgi?request=1&type=p&lastndays=7&dates=since&since=2021-04-01&fromandto=2021-05-01.2021-05-31&level=mlb&franch=ANY
['Name', 'Age', '#days', 'Lev', 'Tm', 'G', 'GS', 'W', 'L', 'SV', 'IP', 'H', 'R', 'ER', 'BB', 'SO', 'HR', 'HBP', 'ERA', 'AB', '2B', '3B', 'IBB', 'GDP', 'SF', 'SB', 'CS', 'PO', 'BF', 'Pit', 'Str', 'StL', 'StS', 'GB/FB', 'LD', 'PU', 'WHIP', 'BAbip', 'SO9', 'SO/W']


In [13]:
headersCounted=['W','IP','H', 'R', 'ER', 'BB', 'SO', 'HR', 'HBP', 'AB', '2B', '3B', 'IBB','Pit', 'Str', 'StL', 'StS']
pDict['Albert Abreu'],len(headersCounted)

([0,
  '8.2',
  '2',
  '2',
  '2',
  '4',
  '10',
  '1',
  '0',
  '26',
  '0',
  '0',
  '0',
  '127',
  '61',
  '20',
  '14'],
 17)

In [14]:
topKWPitching=topKBatting.copy()

for batter, pitcher, date in pitcherInfo:
    if pitcher not in list(pDict.keys()):
        print('pitcher not found: ', pitcher)
        continue
    if batter not in list(topKBatting[:,0]):
        print('batter not found: ', batter)
        continue
    indReq=np.where(topKBatting[:,0]==batter)[0][0]
    print('batter found ', batter, indReq)
    #using data from BvsP to relate batters and pitchers and 
    #assuming we can simply stack them since it's the same day's data for inference
    topKWPitching[indReq,1]=np.hstack((topKWPitching[indReq,1],np.array(pDict[pitcher],dtype=np.float32).reshape(-1,1).squeeze())) 



batter found  Trea Turner 0
batter found  Lourdes Gurriel Jr. 1
batter found  Gregory Polanco 2
batter found  Yermin Mercedes 4
batter found  Patrick Wisdom 6
batter found  Carlos Santana 7
batter found  Robbie Grossman 9
batter found  Jake Fraley 10
batter found  Maikel Franco 12
batter found  Eric Haase 14
batter found  Harold Ramirez 16
batter found  Albert Pujols 17
batter found  Christian Vazquez 18
batter found  Kyle Seager 19
batter found  Charlie Blackmon 20
batter found  Miguel Cabrera 21
batter found  Joey Wendle 22
batter found  Whit Merrifield 23
batter found  Bobby Dalbec 24
batter found  Brandon Lowe 25
batter found  Nick Castellanos 26
pitcher not found:  Vladimir Gutierrez
batter found  Billy McKinney 28
batter found  Matt Chapman 29
batter found  Pete Alonso 30
batter found  Jared Walsh 31
batter found  Javier Baez 33
batter found  Ian Happ 34
batter found  Justin Upton 35
pitcher not found:  Vladimir Gutierrez
batter found  Randal Grichuk 39
batter found  Vladimir Gue

# == PITCHER SECTION == 

In [15]:
topKBatting[1]

array(['Lourdes Gurriel Jr.',
       array([93., 87., 13., 31.,  7.,  0.,  4., 12.,  4.,  0., 22.,  0.,  1.,
        1.,  1.,  0.,  0.], dtype=float32)], dtype=object)

In [16]:
topKBatting[:,0]

array(['Trea Turner', 'Lourdes Gurriel Jr.', 'Gregory Polanco',
       'Gleyber Torres', 'Yermin Mercedes', 'Martin Maldonado',
       'Patrick Wisdom', 'Carlos Santana', 'Gary Sanchez',
       'Robbie Grossman', 'Jake Fraley', 'Yordan Álvarez',
       'Maikel Franco', 'Jose Altuve', 'Eric Haase', 'William Contreras',
       'Harold Ramirez', 'Albert Pujols', 'Christian Vazquez',
       'Kyle Seager', 'Charlie Blackmon', 'Miguel Cabrera', 'Joey Wendle',
       'Whit Merrifield', 'Bobby Dalbec', 'Brandon Lowe',
       'Nick Castellanos', 'Christian Yelich', 'Billy McKinney',
       'Matt Chapman', 'Pete Alonso', 'Jared Walsh', 'Freddie Freeman',
       'Javier Baez', 'Ian Happ', 'Justin Upton', 'Adolis García',
       'Kyle Tucker', 'Avisail Garcia', 'Randal Grichuk',
       'Vladimir Guerrero Jr.', 'Eduardo Escobar', 'Yuli Gurriel',
       'Adam Duvall', 'Teoscar Hernandez', 'Chris Taylor', 'Aaron Judge',
       'Fernando Tatis Jr.', 'Dansby Swanson', 'Eugenio Suarez',
       'Ronald A

In [17]:
# colsReq=list(bVsp.values())[0].shape[0]+topKWPitching[0,1].shape[0]#based on combined columns for stats
colsReq=list(bVsp.values())[0].shape[0]+34 #based on combined columns for stats (already hardcoding 34 anyways)
rowsReq=len(list(bVsp.keys()))#based on no of players actually found for today
fullStats=np.zeros((0,colsReq))
finalPlayerList=[]
for i, found in enumerate(list(bVsp.keys())):    
    req=np.where(topKWPitching[:,0]==found)[0]#line up batters
    if topKWPitching[req][0,1].shape[0]==34 and len(req)>0:
        finalPlayerList.append(found)
        fullStats=np.vstack( (fullStats, np.hstack((topKWPitching[req][0,1],bVsp[found])) ) )
    
    
fullStats

array([[1.24e+02, 1.13e+02, 1.50e+01, ..., 0.00e+00, 0.00e+00, 0.00e+00],
       [9.30e+01, 8.70e+01, 1.30e+01, ..., 6.00e-01, 7.14e-01, 6.00e-01],
       [9.60e+01, 8.40e+01, 7.00e+00, ..., 1.18e-01, 2.86e-01, 1.18e-01],
       ...,
       [8.60e+01, 7.90e+01, 9.00e+00, ..., 1.00e+00, 1.00e+00, 1.00e+00],
       [1.02e+02, 9.50e+01, 5.00e+00, ..., 0.00e+00, 0.00e+00, 0.00e+00],
       [1.00e+02, 9.20e+01, 7.00e+00, ..., 5.00e-01, 5.00e-01, 5.00e-01]])

In [18]:
# b=0
# for i, found in enumerate(list(bVsp.keys())):    
#     req=np.where(topKWPitching[:,0]==found)[0]
    
#     if topKWPitching[req][0,1].shape[0]==34:
#         print(req[0], found, topKWPitching[req,0],finalPlayerList[b])
#         b+=1
# #     elif len(req)>1:
# #         print('more', req[0])
# #     else:
# #         print('00', req[0])

In [19]:
# colsReq=list(bVsp.values())[0].shape[0]+topKBatting[0,1].shape[0]#based on combined columns for stats
# rowsReq=len(list(bVsp.keys()))#based on no of players actually found for today
# fullStats=np.zeros((rowsReq,colsReq))
# finalPlayerList=[]
# for i, found in enumerate(list(bVsp.keys())):    
#     if found in topKBatting[:,0]:
#         finalPlayerList.append(found)
#         fullStats[i]=np.hstack((topKBatting[i,1],bVsp[found]))
    
    
# fullStats

In [20]:
np.save('statsForInference.npy', fullStats)

In [21]:
finalPlayerList
teamsList=[]
for fp in finalPlayerList:    
    teamsList.append([fp, statDict2[fp][3]])
teamsList

[['Trea Turner', 'Washington'],
 ['Lourdes Gurriel Jr.', 'Toronto'],
 ['Gregory Polanco', 'Pittsburgh'],
 ['Yermin Mercedes', 'Chicago'],
 ['Patrick Wisdom', 'Chicago'],
 ['Carlos Santana', 'Kansas City'],
 ['Robbie Grossman', 'Detroit'],
 ['Jake Fraley', 'Seattle'],
 ['Maikel Franco', 'Baltimore'],
 ['Eric Haase', 'Detroit'],
 ['Harold Ramirez', 'Cleveland'],
 ['Albert Pujols', 'Los Angeles'],
 ['Christian Vazquez', 'Boston'],
 ['Kyle Seager', 'Seattle'],
 ['Charlie Blackmon', 'Colorado'],
 ['Miguel Cabrera', 'Detroit'],
 ['Joey Wendle', 'Tampa Bay'],
 ['Whit Merrifield', 'Kansas City'],
 ['Bobby Dalbec', 'Boston'],
 ['Brandon Lowe', 'Tampa Bay'],
 ['Nick Castellanos', 'Cincinnati'],
 ['Billy McKinney', 'Milwaukee,New York'],
 ['Matt Chapman', 'Oakland'],
 ['Pete Alonso', 'New York'],
 ['Jared Walsh', 'Los Angeles'],
 ['Javier Baez', 'Chicago'],
 ['Ian Happ', 'Chicago'],
 ['Justin Upton', 'Los Angeles'],
 ['Randal Grichuk', 'Toronto'],
 ['Vladimir Guerrero Jr.', 'Toronto'],
 ['Eduardo

In [22]:
len(finalPlayerList)

79

In [23]:
import torch
import numpy as np
import os

pList=teamsList
pathReq=os.getcwd() +'/'
stats=np.load(pathReq+'allStats20210609BvsP.npy')

forInference=np.load(pathReq+'statsForInference.npy')
forInference.shape

(79, 46)

In [24]:
def normStats(stats, forInference):
    
    h=['PA','AB','R','H','2B','3B','HR',
           'RBI','BB','IBB','SO','HBP','SH','SF','GDP','SB','CS']#batting only
    # hP=['Dec','IP','H_','R_','ER','BB_','SO_','HR_','HBP_','AB_','2B_','3B_','IBB_','Pit','Str','StL','StS']#had to add underscores to some to separate from h
    hP=['W','IP','H_', 'R_', 'ER_', 'BB_', 'SO_', 'HR_', 'HBP_', 'AB_', '2B_', '3B_', 'IBB_','Pit', 'Str', 'StL', 'StS']
    hBvsP=['PA2','AB2','H2', '1B2','2B2','3B2','HR2','BB2','SO2','AVG2','OBP2','SLG2'] #BvsP
    hTotal=h+hP+hBvsP
    m=dict()
    for i, hh in enumerate(hTotal):
        m[hh]=i
    print(m)
    hTotal=h+hP+hBvsP
    toCount=hTotal

    # toCount=['PA','AB','R','H','2B','3B','HR',
    #          'PA2','AB2','H2', '1B2','2B2','3B2','HR2','BB2','SO2'
    #         ]
    
    finalFeatures=np.zeros((stats.shape[0],len(toCount)))
    finalInference=np.zeros((forInference.shape[0],len(toCount)))     
    
    
    for r in range(finalFeatures.shape[0]):        
        for i, cat in enumerate(toCount):
            if cat in h: # to normalize hitting stats by PA
                if stats[r, m['PA']]>0:
                    finalFeatures[r,m[cat]]=stats[r, m[cat]]/stats[r, m['PA']] 
                else:
                    finalFeatures[r,m[cat]]=stats[r, m[cat]]
            if cat in hP: # to normalize pitching stats by IP
                if stats[r, m['IP']]>0:
                    finalFeatures[r,m[cat]]=stats[r, m[cat]]/stats[r, m['IP']]  
                else:
                    finalFeatures[r,m[cat]]=stats[r, m[cat]]
            if cat in hBvsP: # to normalize BvsP stats by PA2
                if stats[r, m['PA2']]>0:
                    finalFeatures[r,m[cat]]=stats[r, m[cat]]/stats[r, m['PA2']]
                else:
                    finalFeatures[r,m[cat]]=stats[r, m[cat]]
                
    for r in range(finalInference.shape[0]):
        for i, cat in enumerate(toCount):            
            if cat in h: # to normalize hitting stats by PA                
                if forInference[r, m['PA']]>0:
                    finalInference[r,m[cat]]=forInference[r,m[cat]]/forInference[r, m['PA']]
                else:
                    finalInference[r,m[cat]]=forInference[r,m[cat]]
            if cat in hP: # to normalize pitching stats by IP                
                if forInference[r, m['IP']]>0:
                    finalInference[r,m[cat]]=forInference[r,m[cat]]/forInference[r, m['IP']]
                else:
                    finalInference[r,m[cat]]=forInference[r,m[cat]]
            if cat in hBvsP: # to normalize BvsP stats by PA2      
                if forInference[r, m['PA2']]>0:
                    finalInference[r,m[cat]]=forInference[r,m[cat]]/forInference[r, m['PA2']]
                else:
                    finalInference[r,m[cat]]=forInference[r,m[cat]]

    return finalFeatures, finalInference

finalFeatures, finalInference=normStats(stats, forInference)



    

{'PA': 0, 'AB': 1, 'R': 2, 'H': 3, '2B': 4, '3B': 5, 'HR': 6, 'RBI': 7, 'BB': 8, 'IBB': 9, 'SO': 10, 'HBP': 11, 'SH': 12, 'SF': 13, 'GDP': 14, 'SB': 15, 'CS': 16, 'W': 17, 'IP': 18, 'H_': 19, 'R_': 20, 'ER_': 21, 'BB_': 22, 'SO_': 23, 'HR_': 24, 'HBP_': 25, 'AB_': 26, '2B_': 27, '3B_': 28, 'IBB_': 29, 'Pit': 30, 'Str': 31, 'StL': 32, 'StS': 33, 'PA2': 34, 'AB2': 35, 'H2': 36, '1B2': 37, '2B2': 38, '3B2': 39, 'HR2': 40, 'BB2': 41, 'SO2': 42, 'AVG2': 43, 'OBP2': 44, 'SLG2': 45}


In [25]:
finalInference.shape

(79, 46)

In [26]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

features=torch.tensor(finalFeatures) #torch.tensor(stats[:,skippingFrom:skippingTo])
labels=torch.tensor(stats[:,-1])

forInferenceStats=torch.tensor(finalInference) #torch.tensor(forInference[:,skippingFrom:skippingTo])

pctTotal=(1-torch.mean(labels))*2
finalNo=(features.size()[0]*pctTotal).int()

balancedFeatures=torch.zeros(0,features.size()[1])
balancedLabels=torch.zeros(0,1)

c0=0
c1=0

for smpl, l in enumerate(labels):
  if l==1 and c1<(finalNo//2):
    #balancedFeatures[smpl]=features[smpl]
    balancedFeatures=np.vstack((balancedFeatures, features[smpl]))
    balancedLabels=np.vstack((balancedLabels, l))
#     balancedLabels[smpl]=l
    c1+=1
  if l==0 and (c1+c0)<finalNo:
    balancedFeatures=np.vstack((balancedFeatures, features[smpl]))
    balancedLabels=np.vstack((balancedLabels, l))
    c0+=1

   
X_train, X_test, y_train, y_test = train_test_split(torch.tensor(balancedFeatures), torch.tensor(balancedLabels.squeeze()), test_size=0.1, random_state=42)

# X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.1, random_state=42)

scaler = StandardScaler()

x_norm=torch.tensor(scaler.fit_transform(X_train))

x_norm_test=torch.tensor(scaler.fit_transform(X_test))

inference_norm=torch.tensor(scaler.fit_transform(forInferenceStats))


In [27]:
from torch import nn
device='cpu'
net = nn.Sequential(
      nn.Linear(inference_norm.shape[1], 5),
      nn.ReLU(),           
      nn.Linear(5, 10),
      nn.ReLU(),           
      nn.Linear(10, 1),
      nn.Sigmoid(),
      ).to(device)

net=net.double()
#Features46(10-5-1)2021-06-03
#Features46(10-5-20-1)2021-06-10
#Features46(5-10-1)2021-06-11
PATH=pathReq + '/savedModels/Features46(5-10-1)2021-06-11.pth'
net.load_state_dict(torch.load(PATH))

<All keys matched successfully>

In [28]:
with torch.no_grad():
    net.eval()
    net.cpu()
    preds=net(inference_norm)

In [29]:
# test=np.load(pathReq+'allStats20210528BvsP.npy')
# testfeats=test[:,skipping:-1]

# x_norm_test2=torch.tensor(scaler.fit_transform(testfeats))
# y_test2=test[:,-1]

# net.eval()
# net.cpu()
# indices=500
# net(x_norm[:indices].cpu()),y_train[:indices],net(x_norm_test[:indices].cpu()),y_test[:indices]

# # for i in range(indices):
# #   print('pred=', net(x_norm[i].cpu()).item(),'label=', y_train[i])
# found=0
# for i in range(indices):
#   pred_score=net(x_norm_test[i].cpu()).item()
#   if pred_score >0.8:
#     found+=1
#     print('pred test=', pred_score ,'label=', y_test[i])
#     print(found)

In [30]:
preds=[]
with torch.no_grad():
    net.eval()
    net.cpu()
    indices=inference_norm.shape[0]
    for i in range(indices):
        preds.append(net(inference_norm[i].cpu()).item())
    
    preds=torch.tensor(preds)

print(preds)

tensor([0.1308, 0.8059, 0.3052, 0.2355, 0.1443, 0.4810, 0.7708, 0.3333, 0.3302,
        0.9852, 0.0369, 0.2955, 0.2683, 0.1894, 0.9428, 0.4143, 0.8978, 0.8314,
        0.3088, 0.5355, 0.9800, 0.3608, 0.2447, 0.8490, 0.2034, 0.0698, 0.0894,
        0.7733, 0.2370, 0.4544, 0.2506, 0.1906, 0.9002, 0.2570, 0.1854, 0.1667,
        0.2455, 0.4364, 0.2632, 0.8022, 0.4567, 0.1435, 0.8572, 0.1797, 0.8889,
        0.4603, 0.2306, 0.5455, 0.2723, 0.7976, 0.3163, 0.6144, 0.2854, 0.1845,
        0.1900, 0.9022, 0.2941, 0.2196, 0.7301, 0.2264, 0.0656, 0.8654, 0.8122,
        0.8611, 0.8748, 0.8772, 0.6963, 0.9658, 0.4379, 0.8643, 0.3265, 0.3988,
        0.2755, 0.5014, 0.8674, 0.2430, 0.8992, 0.1135, 0.8793])


In [31]:
from datetime import datetime
print(datetime.today().strftime('%Y-%m-%d'))
np.set_printoptions(suppress=True)

nnOut=[]
h=['PA','AB','R','H','2B','3B','HR',
           'RBI','BB','IBB','SO','HBP','SH','SF','GDP','SB','CS']#batting only
hBvsP=['PA2','AB2','H2', '1B2','2B2','3B2','HR2','BB2','SO2','AVG2','OBP2','SLG2'] #BvsP
headersCounted=['W','IP','H', 'R', 'ER', 'BB', 'SO', 'HR', 'HBP', 'AB', '2B', '3B', 'IBB','Pit', 'Str', 'StL', 'StS']
    
topReq=30#preds.shape[0]

for i, ind in enumerate(torch.topk(preds.reshape(1,preds.shape[0]),topReq)[1][0]):  
  nnOut.append(pList[ind])  
  print(pList[ind],torch.topk(preds.reshape(1,preds.shape[0]),topReq)[0][0][i].item(), forInference[ind,-len(hBvsP):])

2021-06-14
['Eric Haase', 'Detroit'] 0.9851832985877991 [2.  2.  1.  1.  0.  0.  0.  0.  1.  0.5 0.5 0.5]
['Nick Castellanos', 'Cincinnati'] 0.9800101518630981 [4.    3.    1.    0.    1.    0.    0.    1.    2.    0.333 0.5   0.667]
['Kyle Schwarber', 'Washington'] 0.965786337852478 [6.    6.    2.    1.    1.    0.    0.    0.    2.    0.333 0.333 0.5  ]
['Charlie Blackmon', 'Colorado'] 0.9427974224090576 [13.    12.     4.     2.     1.     1.     0.     1.     4.     0.333
  0.385  0.583]
['Kris Bryant', 'Chicago'] 0.9022378921508789 [2.  2.  1.  1.  0.  0.  0.  0.  0.  0.5 0.5 0.5]
['Teoscar Hernandez', 'Toronto'] 0.9002020955085754 [4.   4.   1.   0.   1.   0.   0.   0.   3.   0.25 0.25 0.5 ]
['Anthony Rizzo', 'Chicago'] 0.8992215394973755 [2. 1. 1. 1. 0. 0. 0. 1. 0. 1. 1. 1.]
['Joey Wendle', 'Tampa Bay'] 0.8978434801101685 [9.    9.    3.    3.    0.    0.    0.    0.    0.    0.333 0.333 0.333]
['Manuel Margot', 'Tampa Bay'] 0.8889241218566895 [7.    6.    4.    4.    0.    0. 

['Bo Bichette', 'Toronto']

In [382]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(random_state=0, max_iter=1000).fit(x_norm, y_train)

In [383]:
clf.score(x_norm_test, y_test)

0.6864406779661016

In [384]:
from datetime import datetime
print(datetime.today().strftime('%Y-%m-%d'))

lrOut=[]
for i, pred in enumerate(clf.predict_proba(inference_norm)[:,1][np.argsort(-clf.predict_proba(inference_norm)[:,1])][:20]):  
  lrOut.append(pList[i])
  print(pList[i], pred)

2021-06-10
['DJ LeMahieu', 'New York'] 0.9799421557221404
['Dansby Swanson', 'Atlanta'] 0.9501263325202097
['Miguel Andujar', 'New York'] 0.8911460786925804
['Chris Taylor', 'Los Angeles'] 0.8902424361807675
['Kolten Wong', 'Milwaukee'] 0.8719195675588413
['Jonathan India', 'Cincinnati'] 0.860527408582086
['Alex Verdugo', 'Boston'] 0.8582231460256226
['Miguel Cabrera', 'Detroit'] 0.8115514610131283
['Gleyber Torres', 'New York'] 0.8093783353444022
['Jean Segura', 'Philadelphia'] 0.7589783226931003
['Andrelton Simmons', 'Minnesota'] 0.7504267792599948
['Willy Adames', 'Milwaukee,Tampa Bay'] 0.7414021330092617
['Starlin Castro', 'Washington'] 0.7387604513335545
['Trea Turner', 'Washington'] 0.7229193227430409
['Carlos Correa', 'Houston'] 0.6811773576405749
['Kyle Tucker', 'Houston'] 0.6579805861765677
['Brandon Crawford', 'San Francisco'] 0.6487876286612322
['J.P. Crawford', 'Seattle'] 0.6482958070206654
['Yonathan Daza', 'Colorado'] 0.6349026780150395
['Gio Urshela', 'New York'] 0.63199

In [22]:
for o in nnOut:
  if o in lrOut:
    print(o)

['Ramon Laureano', 'Oakland']
['Michael Brantley', 'Houston']
['Cedric Mullins', 'Baltimore']
['Xander Bogaerts', 'Boston']
['Jeimer Candelario', 'Detroit']
['J.D. Martinez', 'Boston']
['David Peralta', 'Arizona']
['Nate Lowe', 'Texas']
['Garrett Cooper', 'Miami']


# ==============================================================
# ==============================================================

In [167]:
allGL=dict()
allGLNumpy=dict()

for p in playersToCheck:
    urlToCheck='https://www.baseball-reference.com/register/'+urlGLDict2[p][len('/minors/'):]
    allGL[p],allGLNumpy[p]=getGLBatter(urlToCheck)    

headersNumpy=['PA','AB','R','H','2B','3B','HR',
               'RBI','BB','IBB','SO','HBP','SH','SF','GDP','SB','CS']

GLTable=dict()

for i, h in enumerate(headersNumpy):
    GLTable[h]=i#just make an easy to use mapping for numpy stats
    
#playersToCheck are the same as allGLNumpy keys


In [168]:
for p in playersToCheck:
    print('Name: ', p, 'GP:', len(allGLNumpy[p][:,GLTable['H']]), 'hit%:', np.mean(allGLNumpy[p][:,GLTable['H']]>0), 'Team:', statDict2[p][3])#find % where batter had a hit    

Name:  Whit Merrifield GP: 33 hit%: 0.7272727272727273 Team: Kansas City
Name:  Kyle Seager GP: 35 hit%: 0.5428571428571428 Team: Seattle
Name:  Avisail Garcia GP: 32 hit%: 0.59375 Team: Milwaukee
Name:  Austin Riley GP: 34 hit%: 0.6176470588235294 Team: Atlanta
Name:  Mitch Haniger GP: 33 hit%: 0.7272727272727273 Team: Seattle
Name:  Colin Moran GP: 32 hit%: 0.65625 Team: Pittsburgh
Name:  Marcus Semien GP: 33 hit%: 0.696969696969697 Team: Toronto
Name:  David Peralta GP: 32 hit%: 0.625 Team: Arizona
Name:  Mike Trout GP: 30 hit%: 0.7333333333333333 Team: Los Angeles
Name:  Carlos Santana GP: 33 hit%: 0.6060606060606061 Team: Kansas City
Name:  Randal Grichuk GP: 33 hit%: 0.7272727272727273 Team: Toronto
Name:  Trevor Story GP: 34 hit%: 0.6764705882352942 Team: Colorado
Name:  Bryan Reynolds GP: 31 hit%: 0.7096774193548387 Team: Pittsburgh
Name:  Miguel Rojas GP: 31 hit%: 0.6451612903225806 Team: Miami
Name:  Justin Turner GP: 33 hit%: 0.7272727272727273 Team: Los Angeles
Name:  Tim A

In [169]:
#===finding required dates to check

playersDates=dict()
datesPossible=[]
for p in playersToCheck:    
    for i, date in enumerate(list(allGL[p].keys())):        
        if "(" not in date:
            datesPossible.append(date)
            
for d in list(set(datesPossible)):
    playersDates[d]=[]
    for p in playersToCheck: 
        if d in list(allGL[p].keys()):
           playersDates[d].append(p)


In [170]:
#getting batter vs pitcher stats from website

bVsp=dict()
listBvsP=[]
for dateReq in list(playersDates.keys()):#loop through every date possible
    print('finding for : ', dateReq)
    
    urlReq='https://swishanalytics.com/optimus/mlb/batter-vs-pitcher-stats?date='+dateReq
    page=requests.get(urlReq)

    soup = BeautifulSoup(page.content, 'html.parser')        

    pgResults=str(soup.find_all('script'))

    #finding way to just get the json type response in our own way
    ss='this.playerArray = '
    i=pgResults.find(ss)
    i2=pgResults.find('}]')
    i2
    dictList=list(eval(pgResults[i+len(ss)+1:i2+len('}')]))
    bVsp[dateReq]=dict()#first key will be date
    
    for p in playersDates[dateReq]:        
        
        for b in dictList:
            if b['batter']==p:        
                pitcherReq=b
                break    

        bVsPHeaders=['pa', 'ab', 'h', 'singles', 'doubles', 
          'triples', 'hr', 'bb', 'so', 'batting_avg', 'obp', 'slugging_pct']

        bVsp[dateReq][p]=np.zeros(len(bVsPHeaders))  #initialize pitching stats array #second key will be player
        
        for hNo, h in enumerate(bVsPHeaders):        
            bVsp[dateReq][p][hNo]=pitcherReq[h]
    
#         listBvsP.append(bVsp)#for each batter add to the list
        print('done: ', p)    

finding for :  2021-04-06
done:  Kyle Seager
done:  Avisail Garcia
done:  Austin Riley
done:  Mitch Haniger
done:  Colin Moran
done:  Marcus Semien
done:  David Peralta
done:  Mike Trout
done:  Randal Grichuk
done:  Trevor Story
done:  Bryan Reynolds
done:  Miguel Rojas
done:  Justin Turner
done:  Vladimir Guerrero Jr.
done:  Jake Cronenworth
done:  Jared Walsh
done:  Raimel Tapia
done:  Xander Bogaerts
done:  Tommy Edman
done:  Bo Bichette
done:  J.D. Martinez
done:  Jeimer Candelario
done:  Alex Verdugo
done:  Giancarlo Stanton
done:  Rafael Devers
done:  Dylan Carlson
done:  Yuli Gurriel
done:  Isiah Kiner-Falefa
done:  Trey Mancini
done:  Nolan Arenado
done:  Starlin Castro
done:  Adam Frazier
done:  Cedric Mullins
done:  DJ LeMahieu
done:  Yordan Álvarez
done:  Trea Turner
done:  Eric Hosmer
done:  Nate Lowe
done:  Nick Solak
done:  David Fletcher
done:  J.P. Crawford
done:  Kris Bryant
finding for :  2021-04-19
done:  Whit Merrifield
done:  Kyle Seager
done:  Avisail Garcia
done:

done:  Whit Merrifield
done:  Kyle Seager
done:  Avisail Garcia
done:  Austin Riley
done:  Mitch Haniger
done:  Colin Moran
done:  Marcus Semien
done:  David Peralta
done:  Mike Trout
done:  Carlos Santana
done:  Randal Grichuk
done:  Trevor Story
done:  Miguel Rojas
done:  Justin Turner
done:  Tim Anderson
done:  Vladimir Guerrero Jr.
done:  Jake Cronenworth
done:  Jared Walsh
done:  Adolis García
done:  Raimel Tapia
done:  Xander Bogaerts
done:  Tommy Edman
done:  Trent Grisham
done:  Salvador Perez
done:  Bo Bichette
done:  J.D. Martinez
done:  Jeimer Candelario
done:  Alex Verdugo
done:  Giancarlo Stanton
done:  Rafael Devers
done:  Dylan Carlson
done:  Jesse Winker
done:  Yuli Gurriel
done:  Isiah Kiner-Falefa
done:  Trey Mancini
done:  Nolan Arenado
done:  Starlin Castro
done:  Adam Frazier
done:  Cedric Mullins
done:  DJ LeMahieu
done:  Yordan Álvarez
done:  Trea Turner
done:  Eric Hosmer
done:  Nate Lowe
done:  Nick Solak
done:  David Fletcher
done:  J.P. Crawford
done:  Kris B

done:  Whit Merrifield
done:  Kyle Seager
done:  Avisail Garcia
done:  Austin Riley
done:  Mitch Haniger
done:  Colin Moran
done:  Marcus Semien
done:  David Peralta
done:  Mike Trout
done:  Carlos Santana
done:  Randal Grichuk
done:  Trevor Story
done:  Bryan Reynolds
done:  Miguel Rojas
done:  Justin Turner
done:  Tim Anderson
done:  Vladimir Guerrero Jr.
done:  Jake Cronenworth
done:  Jared Walsh
done:  Adolis García
done:  Raimel Tapia
done:  Michael Brantley
done:  Xander Bogaerts
done:  Tommy Edman
done:  Trent Grisham
done:  Salvador Perez
done:  Bo Bichette
done:  J.D. Martinez
done:  Jeimer Candelario
done:  Alex Verdugo
done:  Giancarlo Stanton
done:  Rafael Devers
done:  Dylan Carlson
done:  Jesse Winker
done:  Isiah Kiner-Falefa
done:  Trey Mancini
done:  Nolan Arenado
done:  Starlin Castro
done:  Adam Frazier
done:  Cedric Mullins
done:  DJ LeMahieu
done:  Yordan Álvarez
done:  Trea Turner
done:  Eric Hosmer
done:  Nate Lowe
done:  Nick Solak
done:  David Fletcher
done:  J

done:  Whit Merrifield
done:  Kyle Seager
done:  Avisail Garcia
done:  Mitch Haniger
done:  Colin Moran
done:  Marcus Semien
done:  Mike Trout
done:  Carlos Santana
done:  Randal Grichuk
done:  Bryan Reynolds
done:  Miguel Rojas
done:  Justin Turner
done:  Vladimir Guerrero Jr.
done:  Jake Cronenworth
done:  Jared Walsh
done:  Xander Bogaerts
done:  Tommy Edman
done:  Salvador Perez
done:  Bo Bichette
done:  J.D. Martinez
done:  Jeimer Candelario
done:  Alex Verdugo
done:  Giancarlo Stanton
done:  Rafael Devers
done:  Dylan Carlson
done:  Yuli Gurriel
done:  Isiah Kiner-Falefa
done:  Trey Mancini
done:  Nolan Arenado
done:  Adam Frazier
done:  Cedric Mullins
done:  DJ LeMahieu
done:  Yordan Álvarez
done:  Eric Hosmer
done:  Nate Lowe
done:  Nick Solak
done:  David Fletcher
done:  J.P. Crawford
done:  Kris Bryant
finding for :  2021-05-09
done:  Whit Merrifield
done:  Kyle Seager
done:  Avisail Garcia
done:  Austin Riley
done:  Mitch Haniger
done:  Marcus Semien
done:  David Peralta
don

In [171]:
#==adding array for batter vs pitcher
import copy

vsStats=dict()
allGLBvsP=copy.deepcopy(allGL)

for p in playersToCheck:
    vsStats[p]=dict()#might not be needed
    for date in bVsp.keys():#each date for a player        
        if p in list(bVsp[date].keys()):            
            vsStats[p][date]=np.array(bVsp[date][p])
            allGLBvsP[p][date].append(np.array(bVsp[date][p]))
        else:
            continue        

# allGL

In [172]:
def makeNumpyGLWithBvsP(headList, statDict, batter=True,):
  #only getting 'PA','AB','R','H','2B','3B','HR','RBI','BB','IBB','SO','HBP','SH','SF','GDP','SB','CS','bVsP'
    
    h=['PA','AB','R','H','2B','3B','HR',
               'RBI','BB','IBB','SO','HBP','SH','SF','GDP','SB','CS']#batting only
    hBvsP=['PA2','AB2','H2', '1B2','2B2','3B2','HR2','BB2','SO2','AVG2','OBP2','SLG2'] #BvsP

    hTotal=h+hBvsP

    mapping=dict()
    for i, header in enumerate(hTotal):
        mapping[header]=i    
    
    statArray=np.zeros((0,len(h)), dtype=np.float32)#took out string columns
    statArrayBvsP=np.zeros((0,len(hTotal)), dtype=np.float32)#took out string columns +bVsP
    temp=[]#stores required columns only
    
    for k, v in statDict.items():#statDict is player Game Logs k=player, v=stats
        if "(" in k:#make sure date isn't weird like 2021-05-04 (2) and length of array become 16
            continue
        for i in v[4:-3]:# becomes v[4:-3]  instead of v[4:-2] because of extra bVsP stats
            if i != '':
                temp.append(float(i))
            else:
                temp.append(0)
            
        statArray=np.vstack((statArray, np.array(temp, dtype=np.float32)))

        statArrayBvsP=np.vstack( ( statArrayBvsP, np.hstack( ( statArray[-1], np.array(v[-1]) ) ) ) )
        temp=[]
        
    return statArrayBvsP

In [173]:
headersGL=['DatePlayed','Lev','Tm','vs','Opp','PA','AB','R','H','2B','3B','HR',
               'RBI','BB','IBB','SO','HBP','SH','SF','GDP','SB','CS','Pos','E','BvsP']
BvsPWithGL=dict()
for p in playersToCheck:
    BvsPWithGL[p]=makeNumpyGLWithBvsP(headersGL, allGLBvsP[p])
    

In [174]:
# to make it so that the previous games stats appear with batter vs pitcher stats because that's available
#don't run twice
h=['PA','AB','R','H','2B','3B','HR',
           'RBI','BB','IBB','SO','HBP','SH','SF','GDP','SB','CS']#batting only
hBvsP=['PA2','AB2','H2', '1B2','2B2','3B2','HR2','BB2','SO2','AVG2','OBP2','SLG2'] #BvsP

hTotal=h+hBvsP

mapping=dict()
for i, header in enumerate(hTotal):
    mapping[header]=i    

BvsPNew=dict()#just getting batting stats for each player to offset by 1 and accumulate

for p in playersToCheck:
    bStats=BvsPWithGL[p][:,:mapping['CS']+1].copy()#just batting stats
    offset=np.zeros(bStats.shape)#to make new info where info is offset by one
    for game, stats in enumerate(bStats):
        if game==0:#replce first row with 0's
            prevRow=stats.copy()#retain current row 
            offset[game]=np.zeros((1, stats.shape[0]))
        else: #replace rest with previous rows
            offset[game]=prevRow.copy()
        prevRow=stats.copy()#retain current row   
    BvsPNew[p]=offset #input entire offset array of stats for player    
    
    BvsPWithGL[p]=np.hstack( (BvsPWithGL[p], (BvsPWithGL[p][:,3]>0).reshape(-1,1) ) )
    
    for category in h:#make it cumulative stats as well
        colReq=BvsPNew[p][:,mapping[category]]
        BvsPNew[p][:,mapping[category]]=np.cumsum(colReq)
        BvsPWithGL[p][:,mapping[category]]=BvsPNew[p][:,mapping[category]].copy()#also replace required info in original dict
    

In [175]:
BvsPWithGL[playersToCheck[5]][:,-1]

array([0., 1., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1., 1., 1., 0., 1., 1.,
       1., 0., 0., 1., 1., 0., 0., 1., 0., 0., 1., 1., 1., 0.])

In [176]:
#stacking all stats per player

featureLength=BvsPWithGL[playersToCheck[0]].shape[1]
fullDataSet=np.zeros((0, featureLength+1)) # to add binary (hit/no hit) label

for player in list(BvsPWithGL.keys()):
    arrayToStack= np.hstack( (BvsPWithGL[player], (BvsPWithGL[player][:,-1]>0).reshape(-1,1) ) ) #+(BvsPWithGL[p][:,3]>0) for label column
    fullDataSet=np.vstack((fullDataSet,arrayToStack)) 

fullDataSet=np.delete(fullDataSet,BvsPWithGL[player].shape[1]-1, axis=1)#just remove reduntant label column

print('length of data=', fullDataSet.shape[0], 'feature length=', fullDataSet.shape[1])


length of data= 1565 feature length= 30


In [177]:
np.save('allStats.npy',fullDataSet)

loaded=np.load('allStats.npy')

loaded

array([[  0.   ,   0.   ,   0.   , ...,   0.29 ,   0.27 ,   1.   ],
       [  6.   ,   5.   ,   2.   , ...,   0.333,   0.   ,   1.   ],
       [ 11.   ,   9.   ,   4.   , ...,   0.333,   0.667,   1.   ],
       ...,
       [119.   , 103.   ,  22.   , ...,   0.6  ,   0.5  ,   1.   ],
       [123.   , 107.   ,  22.   , ...,   0.5  ,   0.333,   1.   ],
       [127.   , 111.   ,  23.   , ...,   0.5  ,   1.125,   1.   ]])

In [63]:
test=allGLNumpy[p][0]
test=np.hstack((test,np.array([99,99])))

allGLNumpy[p][0].shape, test.shape, allGLNumpy[p][0], test

((17,),
 (19,),
 array([5., 4., 1., 1., 0., 0., 1., 2., 0., 0., 1., 1., 0., 0., 0., 0., 0.],
       dtype=float32),
 array([ 5.,  4.,  1.,  1.,  0.,  0.,  1.,  2.,  0.,  0.,  1.,  1.,  0.,
         0.,  0.,  0.,  0., 99., 99.]))

In [64]:
allGL[p]

{'2021-04-06': ['MLB-NL',
  'Washington Nationals',
  1,
  'Atlanta Braves',
  '5',
  '4',
  '1',
  '1',
  '0',
  '0',
  '1',
  '2',
  '0',
  '0',
  '1',
  '1',
  '0',
  '0',
  '0',
  '0',
  '0',
  'SS',
  '0'],
 '2021-04-07': ['MLB-NL',
  'Washington Nationals',
  1,
  'Atlanta Braves',
  '4',
  '4',
  '1',
  '2',
  '1',
  '0',
  '1',
  '2',
  '0',
  '0',
  '1',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  'SS',
  '0'],
 '2021-04-07 (2)': ['MLB-NL',
  'Washington Nationals',
  1,
  'Atlanta Braves',
  '3',
  '3',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  'SS',
  '0'],
 '2021-04-09': ['MLB-NL',
  'Washington Nationals',
  '@',
  'Los Angeles Dodgers',
  '4',
  '4',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '1',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  'SS',
  '0'],
 '2021-04-10': ['MLB-NL',
  'Washington Nationals',
  '@',
  'Los Angeles Dodgers',
  '5',
  '4',
  '1',
  '1',
  '0',
  '0',
  '0',
  '0',
 

In [37]:
import copy

a=dict()
a['h']=dict()
a['h']['o']='asdasasd'

b = copy.deepcopy(a)

b['h']['o']='ad'

a, b

({'h': {'o': 'asdasasd'}}, {'h': {'o': 'ad'}})

#============================

In [159]:

import sys
import os

if os.getcwd() not in sys.path:
    sys.path.append(os.getcwd())

listBvsP=[]

for p in playersToCheck:
    bVsp=dict()
    bVsp[p]=dict()
    for dateReq in allGL[p].keys():#loop through every game one batter has played    
        if '(' in dateReq:
            continue
        print('finding for : ', dateReq)
        urlReq='https://swishanalytics.com/optimus/mlb/batter-vs-pitcher-stats?date='+dateReq

        page=requests.get(urlReq)

        soup = BeautifulSoup(page.content, 'html.parser')        

        pgResults=str(soup.find_all('script'))

        #finding way to just get the json type response in our own way
        ss='this.playerArray = '
        i=pgResults.find(ss)
        i2=pgResults.find('}]')
        i2
        dictList=list(eval(pgResults[i+len(ss)+1:i2+len('}')]))

        for b in dictList:
            if b['batter']==p:        
                pitcherReq=b
                break
    #     print(pitcherReq)

        bVsPHeaders=['pa', 'ab', 'h', 'singles', 'doubles', 
          'triples', 'hr', 'bb', 'so', 'batting_avg', 'obp', 'slugging_pct']

        bVsp[p][dateReq]=np.zeros(len(bVsP))  #initialize pitching stats array

        playerIndex=0
        for hNo, h in enumerate(bVsPHeaders):        
            bVsp[p][dateReq][hNo]=pitcherReq[h]
    
    listBvsP.append(bVsp)
    print('done: ', p)
    

finding for :  2021-04-01
finding for :  2021-04-02
finding for :  2021-04-03
finding for :  2021-04-04
finding for :  2021-04-06
finding for :  2021-04-07
finding for :  2021-04-08
finding for :  2021-04-09
finding for :  2021-04-10
finding for :  2021-04-11
finding for :  2021-04-13
finding for :  2021-04-14
finding for :  2021-04-15
finding for :  2021-04-17
finding for :  2021-04-18
finding for :  2021-04-20
finding for :  2021-04-21
finding for :  2021-04-23
finding for :  2021-04-24
finding for :  2021-04-25
finding for :  2021-04-26
finding for :  2021-04-27
finding for :  2021-04-28
finding for :  2021-04-29
finding for :  2021-04-30
finding for :  2021-05-01
finding for :  2021-05-02
finding for :  2021-05-04
finding for :  2021-05-05
finding for :  2021-05-07
done:  Trevor Story
finding for :  2021-04-01
finding for :  2021-04-03
finding for :  2021-04-04
finding for :  2021-04-05
finding for :  2021-04-06
finding for :  2021-04-07
finding for :  2021-04-08
finding for :  202

finding for :  2021-04-27
finding for :  2021-04-28
finding for :  2021-04-29
finding for :  2021-04-30
finding for :  2021-05-01
finding for :  2021-05-02
finding for :  2021-05-05
finding for :  2021-05-06
finding for :  2021-05-07
done:  Rafael Devers
finding for :  2021-04-01
finding for :  2021-04-02
finding for :  2021-04-03
finding for :  2021-04-04
finding for :  2021-04-06
finding for :  2021-04-07
finding for :  2021-04-08
finding for :  2021-04-09
finding for :  2021-04-10
finding for :  2021-04-11
finding for :  2021-04-13
finding for :  2021-04-14
finding for :  2021-04-15
finding for :  2021-04-17
finding for :  2021-04-18
finding for :  2021-04-20
finding for :  2021-04-21
finding for :  2021-04-23
finding for :  2021-04-24
finding for :  2021-04-25
finding for :  2021-04-26
finding for :  2021-04-27
finding for :  2021-04-28
finding for :  2021-04-29
finding for :  2021-04-30
finding for :  2021-05-01
finding for :  2021-05-02
finding for :  2021-05-04
finding for :  20

In [183]:
import csv

keys = listBvsP[0].keys()

a_file = open("outputPitchers.csv", "w")
writer = csv.writer(a_file)

for bb in listBvsP:
    for key in bb.keys():
        writer.writerow([key])
        for key2 ,value in bb[key].items():
            writer.writerow([key2, value])

listRead=[]
with open('outputPitchers.csv') as csv_file:    
    reader = csv.reader(csv_file)
    
    for line in reader:
        print(line)
        mydict = dict(line)
        listRead.append(mydict)

['Trevor Story']


ValueError: dictionary update sequence element #0 has length 12; 2 is required

In [71]:
a=[0]*17

a

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [176]:
listBvsP

[{'Trevor Story': {'2021-04-01': array([24.   , 23.   ,  8.   ,  5.   ,  3.   ,  0.   ,  0.   ,  1.   ,
           9.   ,  0.348,  0.375,  0.478]),
   '2021-04-02': array([11.   , 10.   ,  2.   ,  2.   ,  0.   ,  0.   ,  0.   ,  1.   ,
           5.   ,  0.2  ,  0.273,  0.2  ]),
   '2021-04-03': array([31.   , 30.   ,  6.   ,  3.   ,  1.   ,  0.   ,  2.   ,  1.   ,
           7.   ,  0.2  ,  0.226,  0.433]),
   '2021-04-04': array([16.   , 13.   ,  2.   ,  1.   ,  1.   ,  0.   ,  0.   ,  3.   ,
           4.   ,  0.154,  0.313,  0.231]),
   '2021-04-06': array([14.   , 14.   ,  4.   ,  3.   ,  1.   ,  0.   ,  0.   ,  0.   ,
           5.   ,  0.286,  0.286,  0.357]),
   '2021-04-07': array([45.   , 43.   ,  9.   ,  3.   ,  1.   ,  0.   ,  5.   ,  2.   ,
          15.   ,  0.209,  0.244,  0.581]),
   '2021-04-08': array([15.   , 14.   ,  6.   ,  0.   ,  3.   ,  1.   ,  2.   ,  1.   ,
           3.   ,  0.429,  0.467,  1.214]),
   '2021-04-09': array([31.   , 28.   ,  8.   ,  5.   ,  1. 

In [132]:

import sys
import os

if os.getcwd() not in sys.path:
    sys.path.append(os.getcwd())

dateReq='2021-04-03'
urlReq='https://swishanalytics.com/optimus/mlb/batter-vs-pitcher-stats?date='+dateReq

page=requests.get(urlReq)

soup = BeautifulSoup(page.content, 'html.parser')        

pgResults=str(soup.find_all('script'))

#finding way to just get the json type response in our own way
ss='this.playerArray = '
i=pgResults.find(ss)
i2=pgResults.find('}]')
i2
dictList=list(eval(pgResults[i+len(ss)+1:i2+len('}')]))




In [140]:

for b in dictList:
    if b['batter']==p:        
        pitcherReq=b
        break
print(pitcherReq)
bVsP=['pa', 'ab', 'h', 'singles', 'doubles', 
      'triples', 'hr', 'bb', 'so', 'batting_avg', 'obp', 'slugging_pct']

bVsPArray=np.zeros((len(playersToCheck),len(bVsP)))
playerIndex=0
for hNo, h in enumerate(bVsP):
    bVsPArray[playerIndex, hNo]=pitcherReq[h]
            

{'batter': 'Jesse Winker', 'status': '<small class="fantasy-blue"><span class="enhanced-sports enhanced-sports-checkmark-circle2"><\\/span><\\/small>', 'batter_team': 'Reds', 'pitcher': 'Adam Wainwright', 'pitcher_team': 'Cardinals', 'position': 'LF', 'pa': '12', 'ab': '11', 'h': '3', 'singles': '2', 'doubles': '0', 'triples': '0', 'hr': '1', 'bb': '1', 'so': '2', 'batting_avg': '.273', 'obp': '.333', 'slugging_pct': '.546', 'batter_salary_fd': '3000', 'pitcher_salary_fd': '8200', 'batter_salary_dk': '3900', 'pitcher_salary_dk': '7500', 'batter_salary_ya': 'N\\/A', 'pitcher_salary_ya': 'N\\/A', 'fd_pos_batter': 'OF', 'fd_pos_pitcher': 'P', 'dk_pos_batter': 'OF', 'dk_pos_pitcher': 'SP', 'ya_pos_batter': 'LF', 'ya_pos_pitcher': 'LF', 'fpts_fd_ppa': '2.04', 'fpts_dk_ppa': '1.67', 'bats': 'L', 'p_throws': 'R'}


In [143]:
bVsPArray

array([[12.   , 11.   ,  3.   ,  2.   ,  0.   ,  0.   ,  1.   ,  1.   ,
         2.   ,  0.273,  0.333,  0.546],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,
         0.   ,  0.   ,  0.   ,  0.   ],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,
         0.   ,  0.   ,  0.   ,  0.   ],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,
         0.   ,  0.   ,  0.   ,  0.   ],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,
         0.   ,  0.   ,  0.   ,  0.   ],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,
         0.   ,  0.   ,  0.   ,  0.   ],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,
         0.   ,  0.   ,  0.   ,  0.   ],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,
         0.   ,  0.   ,  0.   ,  0.   ],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,
         0.   ,  0.   , 

In [122]:
ss='this.playerArray = '
i=pgResults.find(ss)
i2=pgResults.find('}]')
i2
dictList=list(eval(pgResults[i+len(ss)+1:i2+len('}')]))

In [123]:
dictList

[{'batter': 'Freddie Freeman',
  'status': '<small class="fantasy-blue"><span class="enhanced-sports enhanced-sports-checkmark-circle2"><\\/span><\\/small>',
  'batter_team': 'Braves',
  'pitcher': 'Stephen Strasburg',
  'pitcher_team': 'Nationals',
  'position': '1B',
  'pa': '72',
  'ab': '62',
  'h': '20',
  'singles': '10',
  'doubles': '6',
  'triples': '0',
  'hr': '4',
  'bb': '7',
  'so': '18',
  'batting_avg': '.323',
  'obp': '.389',
  'slugging_pct': '.613',
  'batter_salary_fd': '8500',
  'pitcher_salary_fd': 'N\\/A',
  'batter_salary_dk': 'N\\/A',
  'pitcher_salary_dk': 'N\\/A',
  'batter_salary_ya': 'N\\/A',
  'pitcher_salary_ya': 'N\\/A',
  'fd_pos_batter': '1B',
  'fd_pos_pitcher': '1B',
  'dk_pos_batter': '1B',
  'dk_pos_pitcher': '1B',
  'ya_pos_batter': '1B',
  'ya_pos_pitcher': '1B',
  'fpts_fd_ppa': '2.84',
  'fpts_dk_ppa': '2.14',
  'bats': 'L',
  'p_throws': 'R'},
 {'batter': 'Kolten Wong',
  'status': '',
  'batter_team': 'Brewers',
  'pitcher': 'Kyle Hendricks'

398623

'[{"batter":"Freddie Freeman","status":"<small class=\\"fantasy-blue\\"><span class=\\"enhanced-sports enhanced-sports-checkmark-circle2\\"><\\/span><\\/small>","batter_team":"Braves","pitcher":"Stephen Strasburg","pitcher_team":"Nationals","position":"1B","pa":"72","ab":"62","h":"20","singles":"10","doubles":"6","triples":"0","hr":"4","bb":"7","so":"18","batting_avg":".323","obp":".389","slugging_pct":".613","batter_salary_fd":"8500","pitcher_salary_fd":"N\\/A","batter_salary_dk":"N\\/A","pitcher_salary_dk":"N\\/A","batter_salary_ya":"N\\/A","pitcher_salary_ya":"N\\/A","fd_pos_batter":"1B","fd_pos_pitcher":"1B","dk_pos_batter":"1B","dk_pos_pitcher":"1B","ya_pos_batter":"1B","ya_pos_pitcher":"1B","fpts_fd_ppa":"2.84","fpts_dk_ppa":"2.14","bats":"L","p_throws":"R"},{"batter":"Kolten Wong","status":"","batter_team":"Brewers","pitcher":"Kyle Hendricks","pitcher_team":"Cubs","position":"2B","pa":"49","ab":"45","h":"16","singles":"11","doubles":"3","triples":"0","hr":"2","bb":"3","so":"5","

In [238]:
headersNumpy=['PA','AB','R','H','2B','3B','HR',
               'RBI','BB','IBB','SO','HBP','SH','SF','GDP','SB','CS']

GLTable=dict()

for i, h in enumerate(headersNumpy):
    GLTable[h]=i#just make an easy to use mapping for numpy stats
    
#playersToCheck are the same as allGLNumpy keys
for p in playersToCheck:
    print('Name: ', p, 'GP:', len(allGLNumpy[p][:,GLTable['H']]), 'hit%:', np.mean(allGLNumpy[p][:,GLTable['H']]>0))#find % where batter had a hit

Name:  Carlos Santana GP: 28 hit%: 0.6428571428571429
Name:  Luis Urias GP: 27 hit%: 0.5925925925925926
Name:  Matt Chapman GP: 31 hit%: 0.5161290322580645
Name:  Trea Turner GP: 25 hit%: 0.64
Name:  Willie Calhoun GP: 15 hit%: 0.8666666666666667
Name:  David Peralta GP: 28 hit%: 0.6785714285714286
Name:  Tommy Edman GP: 29 hit%: 0.7931034482758621
Name:  C.J. Cron GP: 28 hit%: 0.5714285714285714
Name:  Ozzie Albies GP: 27 hit%: 0.6296296296296297
Name:  Raimel Tapia GP: 29 hit%: 0.7586206896551724
Name:  Byron Buxton GP: 22 hit%: 0.8181818181818182
Name:  Andrew Benintendi GP: 26 hit%: 0.7692307692307693
Name:  Tyler O'Neill GP: 19 hit%: 0.5789473684210527
Name:  Xander Bogaerts GP: 28 hit%: 0.7857142857142857
Name:  Isiah Kiner-Falefa GP: 31 hit%: 0.6451612903225806
Name:  Trevor Story GP: 30 hit%: 0.7
Name:  Trey Mancini GP: 30 hit%: 0.6333333333333333
Name:  Giancarlo Stanton GP: 25 hit%: 0.68
Name:  Austin Riley GP: 29 hit%: 0.6551724137931034
Name:  Kris Bryant GP: 28 hit%: 0.714

In [175]:
# for pitcher 
URL="https://www.baseball-reference.com/leagues/daily.fcgi?request=1&type=p&dates=lastndays&lastndays=7&since=2021-04-01&fromandto=2021-04-01.2021-04-30&level=mlb&franch=ANY"

page = requests.get(URL)

soup = BeautifulSoup(page.content, 'html.parser')

results=soup.find(id="daily")

In [178]:
cells=results.find_all("td")
headers=results.find_all("thead")
i=0
headListP=[]


for h in headers:
    #print(h.text)
    for cells in h.text.split('\n'):
        if cells.strip() != '' and cells.strip() != 'Rk':
            i+=1        
            headListP.append(cells)
print(headListP)

rows=results.find_all("tr")

statDictP=dict()

for r in rows:
    colCount=0
    statListTemp=[]
    for c in r.find_all("td"):
        
        if c.text.strip()!='gl':
            colCount+=1            
        if colCount==1:
            keyReq=c.text.strip()#using player-name based dictionary
            colCount+=1
        if colCount>1 and c.text.strip()!='gl':
            statListTemp.append(c.text.strip())#input respective stats        
            
    if colCount>1:
        statDictP[keyReq]=statListTemp



['Name', 'Age', '#days', 'Lev', 'Tm', 'G', 'GS', 'W', 'L', 'SV', 'IP', 'H', 'R', 'ER', 'BB', 'SO', 'HR', 'HBP', 'ERA', 'AB', '2B', '3B', 'IBB', 'GDP', 'SF', 'SB', 'CS', 'PO', 'BF', 'Pit', 'Str', 'StL', 'StS', 'GB/FB', 'LD', 'PU', 'WHIP', 'BAbip', 'SO9', 'SO/W']


In [206]:
import numpy as np

def isfloat(value):
  try:
    float(value)    
    return True
  except ValueError:
    return False

#only getting chosen cols
statArrayP=np.zeros((0,len(headListP)-5), dtype=np.float32)
temp=[]
for v in statDictP.values():
    for i in v[4:]:
        if i != '' and isfloat(i[:-1]):#to deal with percentage
            if '%' not in i:
                temp.append(float(i))                
            else:
                temp.append(float(i[:-1])/100)
        else:
            print(i)
            temp.append(0)
    statArrayP=np.vstack((statArrayP, np.array(temp, dtype=np.float32)))
    temp=[]

2
0



2
1
0
1
5
0
1
1
0
0
0
0
0
0
0
3
0



0
0
0
1
6
0
1
9
0
0
0
1
0
0
0
0
3
0



1
0
0
1
2
0
1
0
0
0
1
0
0
0
0
1
1
1


5
2
2
0
4
1
0
1
0
0
0
0
0
0
0

3
0

1

2
1
1
1
2
0
0
1
0
0
0
0
1
0
0
2
0



5
1
1
0
6
1
0
0
0
0
0
0
0
0
0

2
0



6
3
3
1
6
0
0
2
1
0
0
0
1
0
0
1
1

1

5
6
6
1
1
3
0
9
1
0
0
0
0
0
0
0
1
0



2
3
3
3
1
0
0
3
0
0
0
0
0
0
0
0
6
4
0



2
3
3
3
3
0
1
0
0
1
0
0
0
0
0
2
0



2
1
1
1
1
0
0
7
0
0
0
0
0
2
0
0
8
1
1
1


4
2
2
1
6
1
1
1
0
0
0
0
0
0
0
1
1
1


5
1
1
2
6
0
0
3
0
0
0
0
0
0
0
2
2
1
1

7
4
4
3
0
0
2
0
0
0
0
0
0
0
2
2

1

5
3
3
5
0
0
1
0
0
0
1
0
0
0
2
0

1

3
3
3
1
2
1
0
9
0
1
0
0
0
0
0
0
2
0


1
2
4
4
2
1
1
0
7
0
0
0
0
0
0
0
0
9
1
1

1

4
6
6
2
1
4
1
0
0
0
0
0
0
0
0
2
0



1
2
2
2
1
1
0
6
0
0
0
0
1
0
0
0
9
1
1

1

7
7
7
1
2
3
0
1
0
0
0
0
0
0
0
3
0



2
0
0
2
2
0
0
0
0
0
0
0
0
0
0
2
0
1
1

5
5
5
2
0
1
0
2
1
0
0
0
0
0
0
3
0



4
1
1
1
2
0
0
8
2
0
0
0
0
0
0
0
9
2
0
1


0
0
0
1
0
0
0
6
0
0
0
0
0
0
0
0
7
3
0


2
1
0
0
0
5
0
0
8
0
0
0
0
0
0
0
0
8

2
0



1
0
0
0
0
0
1
5
0
0

5
0

1

3
3
3
1
0
1
0
2
0
0
0
0
0
0
0
1
1

1

4
3
3
3
6
0
0
2
0
0
0
0
0
0
0
2
2
1
1

6
6
3
9
1
0
2
1
0
0
0
0
0
0
3
0

1
1
5
5
3
1
1
0
0
1
0
0
0
0
0
0
0
1
1



6
2
2
1
1
1
0
0
0
0
0
0
0
0
0
1
1



5
4
4
2
6
1
1
0
0
0
0
0
0
0
0
1
1

1

4
6
5
2
6
1
0
0
1
0
0
0
0
0
0
3
0



2
0
0
1
4
0
0
1
0
0
0
0
0
0
0
2
0



0
0
0
1
2
0
0
6
0
0
0
0
0
0
0
0
7
4
0

1
2
3
1
1
1
7
1
1
1
0
0
0
0
0
0
1
1
0



3
3
3
0
1
1
0
6
0
0
0
0
0
0
0
0
6

2
0



5
5
5
2
2
1
0
8
2
0
0
0
0
0
0
0
1
1

1

7
8
8
2
1
3
0
1
0
1
0
0
1
0
0
1
1



4
3
3
1
7
1
0
0
0
1
0
0
0
1
0
1
0



2
0
0
0
0
0
0
4
0
0
0
0
0
0
0
0
4

3
0
1


3
2
2
5
3
1
0
1
0
0
0
0
0
0
0
1
0



1
1
1
1
0
0
0
4
1
0
0
0
0
0
0
0
5
2
0

1

1
3
2
2
1
0
0
6
0
0
0
0
0
1
0
0
8
1
1



3
3
3
2
7
1
0
1
0
0
0
0
0
0
0
1
0



1
0
0
2
2
0
0
7
1
0
1
0
0
0
0
0
9
1
1

1

6
5
3
0
5
1
0
0
0
0
0
0
1
0
0

2
0



2
1
1
1
3
0
0
7
1
0
0
0
0
0
0
0
8
1
0



1
0
0
0
2
0
0
7
0
0
0
0
0
0
0
0
7

4
0



7
2
2
0
3
1
1
0
0
0
0
0
0
0
0

3
0



8
8
8
1
2
2
0
0
0
0
0
1
0
0
0
1
1
1


1
1
1
1
7
1
1
0
0

In [204]:
float(2)

2.0

In [202]:
np.argmax(statArrayP[:,0])

0

In [4]:
#print(results.prettify())
GlOnly=results.find_all("a", title="Game Logs")

In [5]:
GlOnly

[<a href="/minors/player.fcgi?id=abreu-009jos&amp;type=bgl" rel="noopener" target="_blank" title="Game Logs">gl</a>,
 <a href="/minors/player.fcgi?id=acuna-002ron&amp;type=bgl" rel="noopener" target="_blank" title="Game Logs">gl</a>,
 <a href="/minors/player.fcgi?id=adames000wil&amp;type=bgl" rel="noopener" target="_blank" title="Game Logs">gl</a>,
 <a href="/minors/player.fcgi?id=adams-001mat&amp;type=bgl" rel="noopener" target="_blank" title="Game Logs">gl</a>,
 <a href="/minors/player.fcgi?id=adrian001ehi&amp;type=bgl" rel="noopener" target="_blank" title="Game Logs">gl</a>,
 <a href="/minors/player.fcgi?id=aguila001jes&amp;type=bgl" rel="noopener" target="_blank" title="Game Logs">gl</a>,
 <a href="/minors/player.fcgi?id=ahmed-000nic&amp;type=bgl" rel="noopener" target="_blank" title="Game Logs">gl</a>,
 <a href="/minors/player.fcgi?id=albert001han&amp;type=bgl" rel="noopener" target="_blank" title="Game Logs">gl</a>,
 <a href="/minors/player.fcgi?id=albies000ozh&amp;type=bgl" rel=

In [140]:
allURLs=[]#https://www.baseball-reference.com/register/player.fcgi?id=fletch002dav&type=bgl
for l in GlOnly:
    allURLs.append('https://www.baseball-reference.com/register/'+l["href"][len('/minors/'):])

stats=requests.get(allURLs[0])
stats=BeautifulSoup(stats.content, 'html.parser')

In [142]:
allURLs[0]

'https://www.baseball-reference.com/register/player.fcgi?id=abreu-009jos&type=bgl'

In [143]:
statTable=stats.find(id="batting_gamelogs_milb")

In [152]:
headersGL=['DatePlayed','Lev','Tm','Opp','PA','AB','R','H','2B','3B','HR',
           'RBI','BB','IBB','SO','HBP','SH','SF','GDP','SB','CS','Pos','E']
statDictGL=dict.fromkeys(headersGL,[])

colNo=0
for table in statTable.find_all('tbody'):#table body
    for row in table.find_all('tr'):#per row        
        for col in row.find_all('td'):            
            #print(col.text)
            #statDictGL[headersGL[colNo]].append(col.text)
            colNo+=1

2021-04-01
MLB-AL
Chicago White Sox
@
Los Angeles Angels
4
4
0
1
0
0
0
0
0
0
2
0
0
0
0
0
0
1B
0
2021-04-02
MLB-AL
Chicago White Sox
@
Los Angeles Angels
5
4
2
2
0
0
1
4
1
0
1
0
0
0
0
0
0
1B
0
2021-04-03
MLB-AL
Chicago White Sox
@
Los Angeles Angels
5
4
0
0
0
0
0
0
1
0
1
0
0
0
1
0
0
1B
0
2021-04-04
MLB-AL
Chicago White Sox
@
Los Angeles Angels
5
3
1
0
0
0
0
0
2
0
3
0
0
0
0
0
0
1B
0
2021-04-05
MLB-AL
Chicago White Sox
@
Seattle Mariners
5
5
0
1
0
0
0
0
0
0
1
0
0
0
0
0
0
1B
0
2021-04-06
MLB-AL
Chicago White Sox
@
Seattle Mariners
5
4
1
1
0
0
1
5
0
0
2
0
0
1
0
0
0
1B
0
2021-04-07
MLB-AL
Chicago White Sox
@
Seattle Mariners
5
5
0
1
0
0
0
0
0
0
2
0
0
0
0
1
0
1B
0
2021-04-08
MLB-AL
Chicago White Sox

Kansas City Royals
4
4
1
1
1
0
0
0
0
0
0
0
0
0
0
0
0
1B
0
2021-04-11
MLB-AL
Chicago White Sox

Kansas City Royals
5
4
0
1
0
0
0
0
1
0
1
0
0
0
0
0
0
1B
0
2021-04-12
MLB-AL
Chicago White Sox

Cleveland Indians
4
3
0
0
0
0
0
0
1
0
2
0
0
0
0
0
0
1B
0
2021-04-13
MLB-AL
Chicago White Sox

Cleveland Ind

In [144]:
# header=statTable.find_all('thead')

# for h in header:    
#     for hh in h.find_all('th'):
#         print(hh)



{'keys': ['DatePlayed',
  'Lev',
  'Tm',
  'Opp',
  'PA',
  'AB',
  'R',
  'H',
  '2B',
  '3B',
  'HR',
  'RBI',
  'BB',
  'IBB',
  'SO',
  'HBP',
  'SH',
  'SF',
  'GDP',
  'SB',
  'CS',
  'Pos',
  'E']}

In [67]:
import pandas as pd
import numpy as np

df=pd.DataFrame(columns=headersGL)
df.set_index('a')

,b
a,


In [57]:
a='video01_00160_frame_160_endo'
a[a.find('frame'):]


'frame_160_endo'

In [70]:
df2

,A,B,C,D,E,F
0,1.0,2013-01-02,1.0,3,test,foo
1,1.0,2013-01-02,1.0,3,train,foo
2,1.0,2013-01-02,1.0,3,test,foo
3,1.0,2013-01-02,1.0,3,train,foo
